In [1]:
import folium
import pickle
import pandas as pd
import networkx as nx
import reverse_geocoder as rg

## Loading the Data

We first load the files that interest us in DataFrames:
- The file `routes` contains route information, and especially the description of the route (i.e. is it for trams, buses, metro, train, etc)
- The file `trips` contains trip information, and especially the route each trip uses
- The file `stop_times` contains all stop information for each trip, especially the stop id
- The file `stop` contains all geographical information for each stop

In [2]:
DATA_PATH = 'data/raw'
PROCESSED_PATH = 'data/processed'
WEBSITE_PATH = 'docs'

In [3]:
routes = pd.read_csv('{}/routes.txt'.format(DATA_PATH), delimiter=',')
routes.head(1)

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type
0,91-10-A-j21-1,37,10,NaN,Tram,900


In [4]:
trips = pd.read_csv('{}/trips.txt'.format(DATA_PATH), delimiter=',')
trips.head(1)

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id
0,91-10-A-j21-1,TA+ej,1.TA.91-10-A-j21-1.1.H,"Ettingen, Bahnhof",10100,0


In [5]:
stop_times = pd.read_csv('{}/stop_times.txt'.format(DATA_PATH), delimiter=',', dtype={'stop_id':'string'})
stop_times.head(1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,120.TA.91-4-H-j21-1.9.R,13:58:00,13:58:00,8503088:0:21,1,0,0


In [6]:
stops = pd.read_csv('{}/stops.txt'.format(DATA_PATH), delimiter=',')
stops.head(1)

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,1100006,"Zell (Wiesental), Bahnhof",47.704632,7.847772,NaN,NaN


In [7]:
# Read the service dates
services = pd.read_csv('{}/calendar.txt'.format(DATA_PATH), delimiter=',')
services[['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']].sum(axis = 0)

monday       15175
tuesday      14905
wednesday    15117
thursday     15479
friday       14916
saturday     11456
sunday       11323
dtype: int64

We can see that the day with the most services is Thursday. We'll therefore keep services that run on that day. That way, we can have a representative overview of a "busy" day on the railway network.

In [8]:
thursday_services = services[services.thursday == 1]

In [9]:
passenger_data = pd.read_excel('{}/peinaussteiger2018.xlsx'.format(DATA_PATH), usecols=[1, 4], names = ['stop_name', 'daily_count'])
passenger_data.head()

,stop_name,daily_count
0,Aarau,37900
1,Aathal,740
2,Aarburg-Oftringen,2500
3,Acla da Fontauna,90
4,Aadorf,1700


## Cleaning the Data

Now that the data is loaded, we're interested in keeping the stops that correspond to modes of transport that interest us, that is, railway transportation. In order to achieve this:
- We filter out the routes that do not interest us
- Subsequently, we filter our the trips using routes that do not interest us
- We remove stops of trips that do not interest us

In [10]:
# Get country info for stops, to keep only the ones in CH
countries = [geo['cc'] for geo in rg.search(list(zip(stops.stop_lat, stops.stop_lon)))]
stops['cc'] = countries

Loading formatted geocoded file...


In [11]:
# Keep trains only
modes_of_interest = [101, 102, 103, 105, 106, 107, 109]

regional_routes = [106, 107, 109]
grandes_lignes = [101, 102, 103, 105]

In [12]:
# Filter and keep routes, trips and stop times that interest us
routes = routes[routes.route_type.isin(modes_of_interest)]

# Keep trips with services on Thursday (busiest day)
trips = trips[trips.route_id.isin(routes.route_id.unique()) & (trips.service_id.isin(thursday_services.service_id))].copy()
trips.drop_duplicates(subset = ['trip_short_name', 'direction_id', 'trip_headsign'], inplace = True)

stop_times = stop_times[stop_times.trip_id.isin(trips.trip_id.unique())].copy()

In [13]:
# Keep stops that are in the filtered stop times and in CH
railway_stops = stops[(stops.stop_id.isin(stop_times.stop_id.unique())) & (stops.cc == 'CH')].copy()

In [14]:
railway_stops[railway_stops.stop_name == 'Lausanne'].head()

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,cc
5800,8501120:0:1,Lausanne,46.516793,6.629091,NaN,8501120P,CH
5801,8501120:0:3,Lausanne,46.516793,6.629091,NaN,8501120P,CH
5802,8501120:0:4,Lausanne,46.516793,6.629091,NaN,8501120P,CH
5803,8501120:0:5,Lausanne,46.516793,6.629091,NaN,8501120P,CH
5804,8501120:0:6,Lausanne,46.516793,6.629091,NaN,8501120P,CH


Notice that some large main stations have multiple stop identifiers: all of these ids share a same prefix. We therefore edit all ids to just include the prefix and then drop duplicates.

In [15]:
# Remove the suffix of the ids of the same stations 
railway_stops['stop_id'] = railway_stops['stop_id'].apply(lambda id_: id_.split(':')[0])
# Cleanup
railway_stops.drop_duplicates(subset=['stop_id'], inplace=True)
railway_stops.drop(columns = ['location_type', 'parent_station'], inplace = True)

In [16]:
railway_stops[railway_stops.stop_name == 'Lausanne']

,stop_id,stop_name,stop_lat,stop_lon,cc
5800,8501120,Lausanne,46.516793,6.629091,CH


In [17]:
print('Initially, there are {} stops when considering all modes of transport.'.format(len(stops)))
print('By considering only railway transport, we are left with {} stops in Switzerland.'.format(len(railway_stops)))

Initially, there are 36448 stops when considering all modes of transport.
By considering only railway transport, we are left with 1663 stops in Switzerland.


In [18]:
# Maintain this dict for ease of retrieval
stop_id_to_name = railway_stops.set_index('stop_id')['stop_name'].to_dict()

In [19]:
# Remove the suffix of the ids of the same stations 
stop_times['stop_id'] = stop_times['stop_id'].apply(lambda id_: str(id_).split(':')[0])
# Remove stop times that correspond to removed stops
stop_times = stop_times[stop_times.stop_id.isin(railway_stops.stop_id)]

# If a train arrives then leaves at the same time from the same stop -> probably a duplicate 
stop_times.drop_duplicates(subset = ['arrival_time', 'departure_time', 'stop_id'], inplace = True)

Let's now clean the passenger data to be able to merge it with the railway stops data. 

In [20]:
print(f'We have passenger frequency data for {len(passenger_data)} stations, while we have a total of {len(railway_stops)} stations in the original dataset.')

We have passenger frequency data for 906 stations, while we have a total of 1663 stations in the original dataset.


In [21]:
passenger_data[passenger_data.daily_count == '<50'].head()

,stop_name,daily_count
18,Alvaneu,<50
28,Altmatt,<50
49,Bernina Lagalb,<50
54,Bernina Suot,<50
66,Biberegg,<50


There are a few cities where the weekly count is `<50`: let's replace that by 50, and set the missing data to have a daily count of 0.

In [22]:
# Add passenger data to railway stops data
# Replace '<50' by 50, and missing data by 0
passenger_data.replace(to_replace = '<50', value = 50, inplace = True)
railway_stops = railway_stops.merge(passenger_data, how = 'left').fillna(0).sort_values('daily_count', ascending = False)
railway_stops.head()

,stop_id,stop_name,stop_lat,stop_lon,cc,daily_count
561,8503000,Zürich HB,47.378176,8.540212,CH,423600.0
1224,8507000,Bern,46.948831,7.439129,CH,184000.0
19,8500010,Basel SBB,47.547413,7.589560,CH,99800.0
1030,8506000,Winterthur,47.500333,8.723818,CH,95100.0
949,8505000,Luzern,47.050176,8.310180,CH,90800.0


Now that we have all the stops corresponding railway stops in Switzerland (i.e. the nodes), we can proceed to create the edges of the desired graph. Group all stop times by the trip id and form a list of stops per trip. Keep only trips that have more than 1 stop. Trips having 1 stop exist because we removed stops outside of Switzerland.

In [23]:
trips_grouped = stop_times.groupby('trip_id')['stop_id'].aggregate(list).reset_index()
trips_grouped = trips_grouped[trips_grouped.stop_id.str.len() > 1]

In [24]:
def make_stop_pairs(stops):
    '''
        Given a list of stop ids, creates a list of tuples
        of consecutive stops
    '''
    pairs = []
    for i in range(1, len(stops)):
        pairs.append((stops[i-1], stops[i]))
    return pairs

In [25]:
# Create edges: corresponds to pairs of consecutive stops
edges = trips_grouped['stop_id'].apply(make_stop_pairs).explode()\
                                .map(lambda pair: pair if pair[0] < pair[1] else (pair[1], pair[0]))
edges_counts = edges.value_counts()

# Build the graph (undirected)
G = nx.Graph()

# Add nodes with weight = passenger frequency if available
for _, row in railway_stops.iterrows():
    G.add_node(row.stop_id, name = row.stop_name, lat = row.stop_lat, lon = row.stop_lon, daily_count = row.daily_count)

# Add edges with weight = number of trains passing on that edge
for edge, count in edges_counts.iteritems():
    G.add_edge(edge[0], edge[1], trains_count = count)

In [26]:
pickle.dump(G, open('{}/railgraph.pickle'.format(PROCESSED_PATH), 'wb'))
pickle.dump(stop_id_to_name, open('{}/stop_id_to_name'.format(PROCESSED_PATH), 'wb'))
edges_counts.to_pickle('{}/edges_counts.pickle'.format(PROCESSED_PATH))
railway_stops.to_pickle('{}/railway_stops.pickle'.format(PROCESSED_PATH))

In [27]:
def scaler(min_, max_, lower, upper):
    return lambda x: (((x - min_) * (upper - lower)) / (max_ - min_)) + lower

In [28]:
edges_scaler = scaler(edges_counts.min(), edges_counts.max(), 0, 6)
nodes_scaler = scaler(railway_stops.daily_count.min(), railway_stops.daily_count.max(), 1, 10)

In [ ]:
# Create map to visualize graph
m = folium.Map(location=[46.771413, 8.471689], zoom_start = 8, tiles='CartoDB Positron', min_zoom=8)

# Draw nodes
node_group = folium.FeatureGroup(name="Railway Stations")    
for node in G.nodes():
    lat, lon = G.nodes[node]['lat'], G.nodes[node]['lon']
    count = G.nodes[node]['daily_count']
    folium.CircleMarker(
        location = [lat, lon],
        popup = '{}: {} pax/day'.format(G.nodes[node]['name'], count), 
        radius = nodes_scaler(count),
        opacity = 0.5,
        fill = True, 
        fillOpacity = 0.5,
        color = '#2C8C99',
        fillColor = '#2C8C99',
    ).add_to(node_group)

# Draw edges
edge_group = folium.FeatureGroup(name="Railroads")
for edge in G.edges():
    points = [(G.nodes[stop]['lat'], G.nodes[stop]['lon']) for stop in edge]
    count = G.edges[edge]['trains_count']
    src_name = G.nodes[edge[0]]['name']
    dst_name = G.nodes[edge[1]]['name']
    folium.PolyLine(points, 
                    color='#C60018', 
                    opacity=1, 
                    weight = edges_scaler(count),
                    popup = '{}-{}: {} per day'.format(src_name, dst_name, count)
   ).add_to(edge_group)

edge_group.add_to(m)
node_group.add_to(m)
folium.LayerControl().add_to(m)
m.save("{}/network.html".format(WEBSITE_PATH))
m